In [3]:
import logging
from os import statvfs as svfs
from re import compile
from bisect import bisect
from pathlib import Path
from prettytable import PrettyTable  # Import PrettyTable

# Configure logging
logging.basicConfig(
    level=logging.ERROR,  # Log only errors and above
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='disk_usage.log',  # Log to a file
    filemode='a'  # Append to the log file
)

class DiskUsage:
    def __init__(self, file):
        try:
            stats = svfs(file)
            self.IO_Block = stats.f_bsize
            self.f_blocks = stats.f_blocks
            self.f_bfree = stats.f_bfree
        except FileNotFoundError:
            logging.error(f"File or path '{file}' not found.")
            raise ValueError(f"File or path '{file}' not found.")
        except PermissionError:
            logging.error(f"Permission denied for accessing '{file}'.")
            raise ValueError(f"Permission denied for accessing '{file}'.")
        except Exception as e:
            logging.exception(f"An unexpected error occurred while accessing '{file}': {e}")
            raise ValueError(f"An error occurred while accessing '{file}': {e}")

    def convert(self, bytes):
        try:
            size = [1, 2**10, 2**20, 2**30, 2**40]
            unit = ['B', 'K', 'M', 'G', 'T']
            index = bisect(size, bytes) - 1
            return f"{bytes / size[index]:.0f}{unit[index]}"
        except Exception as e:
            logging.exception(f"Error converting size: {e}")
            return "Error"

    def size(self):
        try:
            return self.convert(self.f_blocks * self.IO_Block)
        except Exception as e:
            logging.exception(f"Error calculating size: {e}")
            return "Error"

    def free(self):
        try:
            return self.convert(self.f_bfree * self.IO_Block)
        except Exception as e:
            logging.exception(f"Error calculating free space: {e}")
            return "Error"

    def used(self):
        try:
            used = self.f_blocks - self.f_bfree
            return self.convert(used * self.IO_Block)
        except Exception as e:
            logging.exception(f"Error calculating used space: {e}")
            return "Error"

    def usep(self):
        try:
            if self.f_blocks == 0:  # Avoid division by zero
                return "0.00%"
            return f"{(self.f_blocks - self.f_bfree) / self.f_blocks:.0%}"
        except Exception as e:
            logging.exception(f"Error calculating usage percentage: {e}")
            return "Error"

def main():
    try:
        # Compile a regex to filter out unwanted filesystems
        comp = compile(r'^/dev/(?!.*(snap|boot|fuse|docker))')
        # Read mounted filesystems from /proc/mounts
        mline = Path('/proc/mounts').read_text().splitlines()
        fs = [i.split()[:2] for i in mline if comp.search(i)]

        # Create a PrettyTable instance
        table = PrettyTable()
        table.field_names = ["Filesystem", "Size", "Used", "Avail", "Used%", "Mounted on"]

        # Iterate through filesystems and display disk usage
        for x in fs:
            try:
                disk = DiskUsage(x[1])
                table.add_row([x[0], disk.size(), disk.used(), disk.free(), disk.usep(), x[1]])
            except ValueError as e:
                logging.error(f"Error processing {x[1]}: {e}")
                table.add_row([x[0], "Error", "Error", "Error", "Error", x[1]])

        # Print the table
        print(table)
    except FileNotFoundError:
        logging.error("The file '/proc/mounts' was not found.")
        print("Error: The file '/proc/mounts' was not found.")
    except PermissionError:
        logging.error("Permission denied while accessing '/proc/mounts'.")
        print("Error: Permission denied while accessing '/proc/mounts'.")
    except Exception as e:
        logging.exception(f"An unexpected error occurred: {e}")
        print(f"An unexpected error occurred: {e}")

if __name__ == '__main__':
    main()

+-------------------------+-------+------+-------+-------+------------+
|        Filesystem       |  Size | Used | Avail | Used% | Mounted on |
+-------------------------+-------+------+-------+-------+------------+
| /dev/mapper/centos-root |  45G  | 14G  |  31G  |  31%  |     /      |
|     /dev/mapper/vdo1    | 1024G | 32M  | 1023G |   0%  |   /vdofs   |
+-------------------------+-------+------+-------+-------+------------+


In [5]:
from pathlib import Path

D = Path("/root")

files = [ i for i in D.iterdir() if i.is_file()]

print(*files)

/root/.bash_logout /root/mycpgms.tar.bz2 /root/.bash_profile /root/filetrun.go /root/.bashrc /root/fileops.go /root/.cshrc /root/fileinfo.go /root/.tcshrc /root/anaconda-ks.cfg /root/.bash_history /root/.python_history /root/typest.go /root/29-08-2020-US-Function2.ipynb /root/.gitconfig /root/samplefunc.go /root/gifts.py /root/filerename.go /root/03-07-2020-8AM-tuple.ipynb /root/test2.txt /root/fs.py /root/04-01-2021-8AM-US-Close-File 2.ipynb /root/Dockerfile /root/apichanges.py /root/splunk-8.0.4.1-ab7a85abaa98-linux-2.6-x86_64.rpm /root/26-03-2021-US-Batch-Regex2.ipynb /root/31-03-2021-US-Batch-OOPS1.ipynb /root/6-04-2021-US-Batch-Encapulation.ipynb /root/7-04-2021-8-AM-US-Batch-Operator-Overloding.ipynb /root/DNS-Entries /root/texts.go /root/dict1.go /root/29-06-2020-8AM-String-Methods-4.ipynb /root/.vimrc~ /root/.vimrc /root/dict.go /root/28-08-2020-US.ipynb /root/digits.go /root/.viminfo /root/jupyter_notebook_config.py~ /root/30-06-2020-8AM-List.ipynb /root/chefserver-09-07-2020.